<a href="https://colab.research.google.com/github/Fuenfgeld/HospitalNavigator/blob/main/HospitalNavPrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium
from folium import plugins
import pandas as pd

import ipywidgets
import os
import json



In [ ]:
!git clone https://github.com/Fuenfgeld/HospitalNavigator.git


---
## Intitialize Map
---

In [ ]:
# the latitude and Longitude coordinates
UMMlocation = (49.49216583285166, 8.484229445457458)
map_UMM = folium.Map(location = UMMlocation, width = "75%", zoom_start = 17) # max zoom: 18
map_UMM

---
## Display geojson Outline
---
geojson drawing tool: http://geojson.io/

In [ ]:
hauseOutline = 'HospitalNavigator/GeoResources/Emergency.geojson'
display(folium.GeoJson(hauseOutline, name="Emergency").add_to(map_UMM))
display(map_UMM)

---
##Display Ant Path
---

In [ ]:
testGeoJson = 'HospitalNavigator/GeoResources/path/p1toE.geojson'

In [ ]:
def switchPosition(coordinate):
  temp = coordinate[0]
  coordinate[0] = coordinate[1]
  coordinate[1] = temp
  return coordinate

In [ ]:
with open(testGeoJson) as f:
  testWay = json.load(f)

for feature in testWay['features']:
    path = feature['geometry']['coordinates']
finalPath = list(map(switchPosition,path))
finalPath

In [ ]:
path = 'HospitalNavigator/GeoResources/path/p1toE.geojson'
folium.plugins.AntPath([[87.26776471375342,26.81327263862022],
[87.26823423771481,26.812967880844383],
[87.26853302568969,26.813327059565438],
[87.26888059537669,26.81310665457653],
[87.26874949453054,26.812943391345144],
[87.26877998309914,26.812921622896383]]).add_to(map_UMM)
map_UMM

---
## Select Widget
---

In [ ]:
select_widget=ipywidgets.Select(
    options=['Option A', 'Option B'],
    value='Option A',
    description='Select',
    disabled=False)

def selectOption(opt):
    if opt == 'Option A':
        print('A')
    if opt == 'Option B':
        print('B')

ipywidgets.interact(selectOption, opt=select_widget)



---
## Build Navigator Prototype
---




In [ ]:
class navigator:
    def __init__(self):
        self.geoResources = {}
        self.hospitalLocation =(87.26887308978485, 26.812593090218613)
        self.position = 'w'
        self.destination = 'HospitalBillcounter'

        for root, dirs, files in os.walk('HospitalNavigator/GeoResources'):  
            for file in files:
                self.geoResources[file.split('.')[0]] = root+'/'+file

    def changeDestination(self,newDestination):
        self.destination = newDestination
        self.redrawMap()

    def changeStartPoint(self, newStartPoint):
        
        #self.position = newStartPoint #does not work yet
        print(f'Selected Start: {newStartPoint}; Selected Target: {self.destination}')
        #self.redrawMap()
        

    def drawPathWay(self,hospitalMap):
      
      def switchPosition(coordinate):
        temp = coordinate[0]
        coordinate[0] = coordinate[1]
        coordinate[1] = temp
        return coordinate

      searchString = self.position + self.destination.split('Haus')[1]
      with open(self.geoResources[searchString]) as f:
           testWay = json.load(f)

      for feature in testWay['features']:
        path = feature['geometry']['coordinates']

      finalPath = list(map(switchPosition,path))
      folium.plugins.AntPath(finalPath).add_to(hospitalMap)

    def drawBuilding(self,hospitalMap):
      hauseOutline = self.geoResources[self.destination]
      folium.GeoJson(hauseOutline, name="geojson").add_to(hospitalMap)

    def redrawMap(self):
        #print(f'position {self.position}, destination {self.destination}')
        hospitalMap = folium.Map(location = self.hospitalLocation, width = "75%", zoom_start = 17)
        self.drawPathWay(hospitalMap)
        self.drawBuilding(hospitalMap)
        display(hospitalMap)




In [ ]:
myNavigator = navigator()

In [ ]:
def displayWay(whereTo):
     myNavigator.changeDestination(whereTo)

In [ ]:
def changePosition(whereFrom):
    myNavigator.changeStartPoint(whereFrom)

In [ ]:
# Destination Selector
selectHouse_widget=ipywidgets.Select(
    
options=['Emergency',
    'OPD',
    'BillCounter',
    'Hospital Pharmacy',
    'Hospital Canteen'],
    value='Emergency',
    description='Target',
    disabled=False)

# widget function
def selectHouse(way):
    if way == 'p1toE' :
        displayWay('Emergency' ) 
    if way == 'p1toOPD':
        displayWay('OPD')
    if way == 'p1toHC':
        displayWay('HospitalCanteen')
    if way == 'p1toHP':
        displayWay('Hospital Pharmacy')
    if way == 'p1toBC':
        displayWay('BillCounter')

# Position Selector
selectPosition_widget=ipywidgets.Select(
    options=['Aufnahme', 'Eingang West', 'Eingang Ost', 'Eingang Fahrzeuge'],
    value='Eingang West',
    description='Start',
    disabled=False)

def selectPosition(position):
    if position == 'Aufnahme':
        changePosition('a')
    if position == 'Eingang West':
        changePosition('w')
    if position == 'Eingang Ost':
        changePosition('o')
    if position == 'Eingang Fahrzeuge':
        changePosition('f')
        
#Initialization   
ipywidgets.interact(selectPosition, position=selectPosition_widget)
ipywidgets.interact(selectHouse, way=selectHouse_widget)